## What is DN..{hmmm..}...C? 

* Differentiable Neural Computer is an extension  to NTM. 
* The main difference is read and write part is more unrestricted in this. 
* Let's have a look at this guy

<img src='DNC_Architecture.png' border=5 >

##### [1]. Interaction between heads and Memory

* The head uses 3 distinct forms of interaction:
    1. <b><u>Content lookup</b></u>:
        * The controller emits a key, which is compared to all the memory cells to find the most similiar content using a similiarity method, like cosine distance.
        * The similiarity scores determine an associative recall for a read head or modifying an existing vector by a write head.
        * Importantly, a key that only partially matches the content of a memory location can still be used to attend strongly to that location.        
        * This enables a form of pattern completion, whereby the value recovered by reading the memory location includes additional information that's not present in the key.        
        * In general, key-value retrieval provides a  rich mechanism for navigating associative datastructures in external memory, because the content of one address can effectively encode references to other addresses.<br>

    2. <b><u>Records Transition at write:</b></u>
        * A second attention mechanism, records transition between consecutively written locations in an $N \times N$ temporal link matrix $L$.
        
        * $L[i,j] $ is close to 1 if '$i$' was the next location, to be written after '$j$'  and is close to 0 otherwise.
        
        * For any weighting $\boldsymbol w$ , the operation $ L \boldsymbol w$ smoothly shifts the focus forward to the locations written after those emphasized in $\boldsymbol w$, whereas $L^ \boldsymbol T \boldsymbol w$ , shifts the focus backwards.
        
        * This enables the DNC to recover the sequence in the order that it wrote them, even when consecutive writes didnot occur in adjacent time-step.
        
    3. <b><u>Memory for writing :</b></u>
    
        * The third form of attention, allocates memory for writing.
        
        * The usage of each location is represented as a number between 0 and 1, and a weighting that picks out unused locations is delivered to the write head.
        
        * As well as automatically increasing memory after each write to a location,usage can be decreased  after each read.
        * This allows the controller to re-allocate memory that is no longer required.
        
        * The allocation is independent of the size or content of the memory. What this means is DNC can be trained to solve a task  using one type of memory and later upgraded to a larger memory without retraining.
        
        * In principle, this would let it automatically increase the number of locations over an unbounded external memory, whenever the allocated number crosses a threshold.
        

### Methods and equations in supplementary paper 

### Controller Network

* At time $t$ , the input received by the controller $\mathcal N$ is given by $\boldsymbol x_t \in \mathbb R^X$ and emits an output vec $\boldsymbol y_t \in \mathbb R^Y$ , that parameterizes  a predictive distribution for a target vector $\boldsymbol z_t \in \mathbb R^Y$

* Additionally the controller also receives a set of $\boldsymbol R$ read vectors $\boldsymbol r_{t-1}^1, .. ,\boldsymbol r_{t-1}^R $ from the memory matrix, $M_{t-1} \in \mathbb R^{N \times W}$, at the previous time-step , via the read heads.

* It then emits an interface vector , $\boldsymbol \xi_t$, which defines it's interactions with the memory at the current time-step.

* For notational convenience, we concatenate the read and input vectors to obtain a single controller input vector $\boldsymbol \chi_t=[\boldsymbol {x_t;r_{t-1}^1;r_{t-1}^2;..;r_{t-1}^R}]$

* Although any NN can be used , the following LSTM conf is used:

$$\color{blue}{input:}$$

$$\boldsymbol {i_t^l = \sigma(W_i^l[\chi_t;h_{t-1}^l;h_t^{l-1}]+b_i^l)}$$

$$\color{blue}{forget:}$$

$$\boldsymbol{f_t^l = \sigma(W_f^l[\chi_t;h_{t-1}^l;h_t^{l-1}]+b_f^l)}$$

$$\color{blue}{state:}$$

$$\boldsymbol{s_t^l = f_t^l s_{t-1}^l + i_t^l tanh(W_s^l[\chi_t;h_{t-1}^l;h_t^{l-1}]+b_s^l)}$$

$$\color{blue}{output-gate:} $$

$$\boldsymbol{o_t^l = \sigma(W_o^l[\chi_t;h_{t-1}^l;h_t^{l-1}]+b_o^l)}$$

$$\color{blue}{hidden:}$$

$$\boldsymbol{h_t^l = o_t^l tanh(s_t^l)}$$


* $\qquad$where l - layer index, t- time, $h_t^0=0,$ for all $t$ $h_0^l = s_0^l =0 $ for all $l$

* At each time step, the controller emits an output vector  $\nu_t$ and an interface vector $\boldsymbol{\xi_t \in \mathbb R_t^{(W \times R)+3W+5R+3}}$ , defined as :

$$\boldsymbol {\nu_t = W_y[h_t^1;..;h_t^L]}$$
$$\boldsymbol {\xi_t = W_\xi [h_t^1;..;h_t^L]}$$

* Assuming the controller network is recurrent, it's outputs are a function of the complete history $\boldsymbol{(\chi_1,..\chi_t)}$ of it's input upto current time-step.


* We can therefore encapsulate the operation  of the controller as :
    $$\boldsymbol{(\nu_t,\xi_t)= \mathcal N  ([\chi_1..\chi_t];\theta)} $$
    
    $\qquad\qquad\qquad\qquad$where $\boldsymbol \theta$ is the set of trainable network weights

* Finally , the output vector $\boldsymbol{y_t}$ is defined by adding $\nu_t$ to a vector obtained by passing the concatenation of the current read vectors through the $\mathtt {\boldsymbol{RW \times Y}}$ weight matrix $\boldsymbol{W_r}$

$$\boldsymbol {y_t = \nu_t + W_r[r_t^1,..,r_t^R]}$$


* This arrangement actually helps DNC condition it's output decisions on memory that has just been read. It would not be possible to pass this information back to controller, and thereby use them to determine '$\nu$' , without creating a cycle in the computation graph.

